In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from BalanceDataset import balancedWithRandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, LSTM, Flatten, Dropout, Reshape
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
df = pd.read_csv("cleaned_dataset_dl.csv")
X = df.drop("Class", axis=1)
y = df["Class"]

In [5]:
df_cleaned = df.dropna(subset=['Class'])
X = df_cleaned.drop("Class", axis=1)
y = df_cleaned["Class"]


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
X_train_bal, y_train_bal = balancedWithRandomOverSampler(X_train_scaled, y_train)


Non-Frauds: 16562 / 50.0 % of the dataset
Frauds: 16562 / 50.0 % of the dataset


In [8]:
X_train_reshaped = X_train_bal.reshape(X_train_bal.shape[0], X_train_bal.shape[1], 1)
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


In [9]:
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall(), AUC()])


In [11]:
es = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(X_train_reshaped, y_train_bal, validation_split=0.2,
                    epochs=20, batch_size=128, callbacks=[es], shuffle=True)


Epoch 1/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - auc: 0.8818 - loss: 0.4062 - precision: 0.7063 - recall: 0.8129 - val_auc: 0.0000e+00 - val_loss: 0.0099 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - auc: 0.9973 - loss: 0.0594 - precision: 0.9700 - recall: 0.9753 - val_auc: 0.0000e+00 - val_loss: 9.0777e-04 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - auc: 0.9992 - loss: 0.0184 - precision: 0.9927 - recall: 0.9961 - val_auc: 0.0000e+00 - val_loss: 1.0306e-04 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - auc: 0.9999 - loss: 0.0090 - precision: 0.9957 - recall: 0.9970 - val_auc: 0.0000e+00 - val_loss: 0.0013 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 5/20
208/208 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - auc: 0.9994 - loss: 0.0157 - precision: 0.9940 - recall: 0.9947 - val_auc: 0.0000e+00 - val_loss: 4.5411e-04 - val_p

In [12]:
loss, precision, recall, auc = model.evaluate(X_test_reshaped, y_test)
f1 = 2 * (precision * recall) / (precision + recall)
print("📊 Final Scores:")
print(f"Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f} | AUC: {auc:.4f}")


173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - auc: 0.7228 - loss: 0.0105 - precision: 0.1263 - recall: 0.6314
📊 Final Scores:
Precision: 0.2222 | Recall: 0.8571 | F1: 0.3529 | AUC: 0.9980


In [13]:
model.save("fraud_model.h5")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']